<a href="https://colab.research.google.com/github/jmhuer/shift_invariant_dictionary_learning/blob/main/drum_encoder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install skorch
!pip install tqdm

# Plot Utils 

In [ ]:
import plotly.graph_objects as graph
def plot(all_history:list, title:str, log = False):
    """
    input:
        all_history: list of dicts to plot
    ret:
        None: show plotly fig
    """
    fig = graph.Figure(layout = graph.Layout(title=graph.layout.Title(text=title))) 
    for i in range(len(all_history)):
        fig.add_trace(graph.Scatter(x = all_history[i]["x"], 
                                    y = all_history[i]["y"],
                                    name = all_history[i]["legend"])) 
    if log: fig.update_xaxes(type="log")
    fig.show()

# Generate some drum synthetic data similar to symbolic music

In [28]:
# lets make the data 400 in lenghth to match autoencoder imlementation 
#lets have 10 different sections each 40 in lenght -- ideal conditions 

import numpy as np

## basic patters
downbeat = [1 if i % 4 ==0 else 0 for i in range(0,40)]
downbeat2x = [1 if i % 2 ==0 else 0 for i in range(0,40)]

high_hats = [2 if i % 4 ==1 else 0 for i in range(0,40)]
high_hats2x = [2 if i % 2 ==1 else 0 for i in range(0,40)]

tom_drum = [3 if i % 4 ==2 else 0 for i in range(0,40)]

## combine basic patters

comb1 = np.array(downbeat) + np.array(high_hats)  
comb2 = np.array(downbeat) + np.array(high_hats2x)  
comb3 = np.array(downbeat2x) + np.array(high_hats2x)  
comb4 = np.array(downbeat) + np.array(high_hats2x) + np.array(tom_drum)
comb5 = np.array(downbeat2x) + np.array(high_hats)


print(downbeat)
print(downbeat2x)
print(high_hats)
print(high_hats2x)
print(tom_drum, "\n")

print(list(comb1))
print(list(comb2))
print(list(comb3))
print(list(comb4))
print(list(comb5))


[1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0]
[1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0]
[0, 2, 0, 0, 0, 2, 0, 0, 0, 2, 0, 0, 0, 2, 0, 0, 0, 2, 0, 0, 0, 2, 0, 0, 0, 2, 0, 0, 0, 2, 0, 0, 0, 2, 0, 0, 0, 2, 0, 0]
[0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2]
[0, 0, 3, 0, 0, 0, 3, 0, 0, 0, 3, 0, 0, 0, 3, 0, 0, 0, 3, 0, 0, 0, 3, 0, 0, 0, 3, 0, 0, 0, 3, 0, 0, 0, 3, 0, 0, 0, 3, 0] 

[1, 2, 0, 0, 1, 2, 0, 0, 1, 2, 0, 0, 1, 2, 0, 0, 1, 2, 0, 0, 1, 2, 0, 0, 1, 2, 0, 0, 1, 2, 0, 0, 1, 2, 0, 0, 1, 2, 0, 0]
[1, 2, 0, 2, 1, 2, 0, 2, 1, 2, 0, 2, 1, 2, 0, 2, 1, 2, 0, 2, 1, 2, 0, 2, 1, 2, 0, 2, 1, 2, 0, 2, 1, 2, 0, 2, 1, 2, 0, 2]
[1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2]
[1, 2, 3, 2, 1, 2, 3, 2, 1, 2,

# Auto Econder

In [ ]:
import torch
import torch.optim as optim
torch.manual_seed(42)

class autoencoder(torch.nn.Module):
    def __init__(self):
        super(autoencoder, self).__init__()
        self.encoder = torch.nn.Conv1d(in_channels=1, out_channels=10, kernel_size=40, padding=0, bias=False, stride=40)
        self.decoder = torch.nn.ConvTranspose1d(in_channels=10, out_channels=1, kernel_size=40, padding=0, bias=False, stride=40)
    def get_kernels(self):
        return self.encoder.weight.data[:,0,:]
    def feature_map(self, x):
        code = self.encoder(x)
        return code
    def forward(self, x):
        code = self.encoder(x)
        reconstruct = self.decoder(code)
        return reconstruct



model = autoencoder()
inputs_size = torch.tensor(X_train).float()
out_size = model(inputs)

print(inputs_size.size())
print(out_size.size())

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

loss_fn = torch.nn.L1Loss().to(device)
optimizer = optim.SGD(model.parameters(), lr=.05, weight_decay = 0.00001, momentum=0.05) ##this has weight decay just like you implemented
print(inputs.size())

epochs = 12000
history = {"loss": []}
for i in range(epochs):
  optimizer.zero_grad()
  output = model(inputs)
  # print(output.size())
  loss = loss_fn(output, inputs)
  loss.backward()
  optimizer.step()
  history["loss"].append(float(loss))
  if i % 1000 == 0: print("Epoch : {} \t Loss : {} \t ".format(i, round(float(loss),4)))


